Demonstrate how to use this design.  A single AXI Direct Memory Access has send and receive channels enabled.  After receiving a buffer, it is routed to the AXI4-Stream Data FIFO, which routes it to the send channel.

On the PS side, the class DMADemo defines shared memory fixed to the transfer size of an int16 array.

This notebook demonstration has these steps:
* Define the class:  use the largest transferSize that works.
* Send and receive one time:  define unique values in the array, send it, receive it, and verify that the values are identical
* Send and receive multiple times:  change the values each time and verify values
* Estimate bandwidth:  Eliminate calls to np.array_equiv to meaure "only" transfers.  On the ZCU111 this gets an aggregate speed (write+read) of 56 M Bytes/sec.
* Demonstrate FIFO:  use a small array_size and send 100 unique arrays.  Then, read them bac and check values.

If transferSize is 2**15 or larger, you get an error like this:

ValueError: Transfer size is 67108864 bytes, which exceeds the maximum DMA buffer size 67108863.



In [ ]:
from pynq import Overlay, allocate
import numpy as np
import datetime

### Define the class

In [ ]:
class DMADemo:
    def __init__(self, transferSize=2**14):
        self.overlay = Overlay('DMADemo.bit')
        self.send = self.overlay.dma.sendchannel
        self.recv = self.overlay.dma.recvchannel
        self.sb = allocate(shape=(transferSize,), dtype=np.int16)
        #self.rb = allocate(shape=(transferSize,), dtype=np.int16)
        self.rb = self.sb
        self.transferSize = transferSize
    def sendData(self,data):
        if len(data) != self.transferSize:
            raise ValueError("len(data)=%d is not transferSize=%d"%(len(data),self.transferSize))
        np.copyto(self.sb, data)
        self.send.transfer(self.sb)
        
    def recvData(self):
        self.recv.transfer(self.rb)
        data = np.zeros(self.transferSize, dtype=np.int16)
        np.copyto(data, self.rb)
        return data

#dd = DMADemo(2**25) # sendData does not work at this size
dd = DMADemo(2**25-1)


### Send and receive one array

In [ ]:
# Send one array and receive it back
data_size = dd.transferSize
print("data_size =",data_size, "  nBytes =",2*data_size)
dSend = 2*np.arange(data_size, dtype=np.int16) + 9876
dd.sendData(dSend)
dRecv = dd.recvData()
# Confirm that the same numbers come back
assert(np.array_equiv(dSend, dRecv))
# And this is not just sharing
dSend[0]  = 111
assert(not np.array_equiv(dSend, dRecv))
print(type(dRecv[0]))

### Send and receive multiple times

In [ ]:
#data_size = 10000
#dd = DMADemo(data_size)
nSend = 10
t0 = datetime.datetime.now()
for iSend in range(nSend):
    print(".",end="")
    dSend = (2*np.arange(data_size) + 9876*iSend).astype(np.int16)
    #dSend = np.zeros(data_size, dtype=np.int16) - iSend
    dd.sendData(dSend)
    #time.sleep(0.1)
    dRecv = dd.recvData()
    # Confirm that the same numbers come back
    assert(np.array_equiv(dSend, dRecv))
    # And this is not just sharing
    dSend[0]  = 111
    assert(not np.array_equiv(dSend, dRecv))
t1 = datetime.datetime.now()
seconds = (t1-t0).total_seconds()
print("\nSuccess", seconds)

### Estimate bandwidth

In [ ]:
nSend = 10
t0 = datetime.datetime.now()
for iSend in range(nSend):
    print(".",end="")
    dSend = (2*np.arange(data_size) + 9876*iSend).astype(np.int16)
    dd.sendData(dSend)
    dRecv = dd.recvData()
t1 = datetime.datetime.now()
seconds = (t1-t0).total_seconds()
print("\nSuccess", seconds)
rate = nSend*2*data_size/seconds
print("Rate = %.2e Bytes/sec"%rate)

### Demonstrate FIFO

In [ ]:
# There is a FIFO in the block design, so you can send a few packets and they come one in order

dd = DMADemo(16)
data_size = dd.transferSize
nSends = 100
dSends = []
for i in range(nSends):
    dSend = ((i+1)*np.arange(data_size) + 9876).astype(np.int16)
    
    try:
        dd.sendData(dSend)
        dSends.append(dSend)
    except RuntimeError:
        nBytes = 2*(i-1)*data_size
        print("nBytes=%d"%nBytes)
        break
print("\nlen(dSends) =",len(dSends))    
for i in range(len(dSends)):
    dRecv = dd.recvData()
    assert(np.array_equiv(dSends[i], dRecv))
    print(i, end=" ")
print("\nSuccess")